In [ ]:
import os
import torch
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


# ===============================
# 1. LOAD MODEL & TOKENIZER (LoRA)
# ===============================
def load_model(model_name="Qwen/Qwen3-0.6B", device="cuda"):
    print(f"[INFO] Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    
    # Thêm pad token nếu chưa có
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        device_map="auto",
        torch_dtype=torch.float16
    )

    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)
    print("[INFO] LoRA layers attached successfully.")
    model.print_trainable_parameters()
    return model, tokenizer


# ===============================
# 2. LOAD & PREPROCESS DATA - FIXED PADDING
# ===============================
def load_and_prepare_data(tokenizer, split_ratio="train[:3%]"):
    print("[INFO] Loading dataset...")
    ds = load_dataset("OpenHust/vietnamese-summarization", split=split_ratio)
    
    text_column = 'Document'
    summary_column = 'Summary'
    
    print(f"[INFO] Dataset size: {len(ds)} samples")

    def preprocess_function(examples):
        texts = []
        
        for i in range(len(examples[text_column])):
            input_text = examples[text_column][i]
            summary_text = examples[summary_column][i]
            
            # Tạo một chuỗi duy nhất: input + summary
            full_text = f"### Input:\n{input_text}\n\n### Summary:\n{summary_text}{tokenizer.eos_token}"
            texts.append(full_text)
        
        # Tokenize với padding để tất cả sequences có cùng độ dài
        tokenized = tokenizer(
            texts,
            max_length=1024,  # Giới hạn độ dài
            truncation=True,
            padding=True,     # QUAN TRỌNG: Thêm padding
            return_tensors=None
        )
        
        # Labels chính là input_ids
        tokenized["labels"] = tokenized["input_ids"].copy()
        
        return tokenized

    tokenized_ds = ds.map(
        preprocess_function,
        batched=True,
        batch_size=1000,
        remove_columns=ds.column_names,
        num_proc=1
    )

    # Phân tích kết quả tokenization
    input_lengths = [len(x['input_ids']) for x in tokenized_ds]
    
    print(f"\n📊 TOKENIZATION STATS:")
    print(f"Total tokens - Avg: {sum(input_lengths)/len(input_lengths):.1f}, Max: {max(input_lengths)}, Min: {min(input_lengths)}")
    
    # Kiểm tra xem tất cả sequences có cùng độ dài không
    unique_lengths = set(input_lengths)
    print(f"Unique sequence lengths: {unique_lengths}")
    
    return tokenized_ds


# ===============================
# 3. TRAINING SETUP - FIXED
# ===============================
def setup_training_args(OUTPUT_DIR="./runs/qwen3-0.6B-summarization", NUM_EPOCHS=3):
    os.makedirs("runs", exist_ok=True)
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=1e-4,
        warmup_ratio=0.1,
        fp16=True,
        logging_steps=10,
        save_strategy="epoch",
        save_total_limit=2,
        optim="paged_adamw_32bit",
        report_to="none",
        remove_unused_columns=True,  # Đặt True để trainer tự xử lý
        dataloader_pin_memory=False,
        gradient_checkpointing=True,
        max_grad_norm=0.3,
        dataloader_num_workers=0,
        # Thêm cấu hình để tránh lỗi padding
        group_by_length=False,  # QUAN TRỌNG: Không group by length
    )
    return training_args


# ===============================
# 4. DATA COLLATOR - SIMPLIFIED
# ===============================
def create_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        pad_to_multiple_of=8,
    )


# ===============================
# 5. TRAIN FUNCTION - IMPROVED DEBUG
# ===============================
def train_model(model, tokenizer, dataset, training_args, SAVE_DIR="./runs/qwen3-0.6B-summarization-lora"):
    print("[INFO] Initializing Trainer...")

    data_collator = create_data_collator(tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    print("[INFO] Start training with LoRA...")
    
    # Kiểm tra dataset trước
    print("[DEBUG] Dataset info:")
    print(f"  - Number of samples: {len(dataset)}")
    print(f"  - Features: {dataset.features}")
    
    # Kiểm tra một sample
    print("[DEBUG] First sample structure:")
    first_sample = dataset[0]
    for key in first_sample.keys():
        print(f"  - {key}: {type(first_sample[key])}, length: {len(first_sample[key])}")
    
    # Kiểm tra data collator
    print("[DEBUG] Testing data collator with 2 samples...")
    test_samples = [dataset[0], dataset[1]]
    try:
        collated_batch = data_collator(test_samples)
        print(f"[DEBUG] Collated batch shapes:")
        for key, value in collated_batch.items():
            print(f"  - {key}: {value.shape}")
    except Exception as e:
        print(f"[ERROR] Data collator failed: {e}")
        return None

    # Kiểm tra dataloader
    print("[DEBUG] Checking dataloader...")
    try:
        train_dataloader = trainer.get_train_dataloader()
        sample_batch = next(iter(train_dataloader))
        
        print(f"[DEBUG] Batch shapes:")
        print(f"  - Input IDs: {sample_batch['input_ids'].shape}")
        print(f"  - Labels: {sample_batch['labels'].shape}")
        print(f"  - Attention mask: {sample_batch['attention_mask'].shape}")
        
        # Kiểm tra nội dung
        print(f"[DEBUG] Sample input (first 30 tokens):")
        print(tokenizer.decode(sample_batch['input_ids'][0][:30]))
        
    except Exception as e:
        print(f"[ERROR] Dataloader failed: {e}")
        return None

    # Bắt đầu training
    try:
        result = trainer.train()
        
        # Lưu model
        save_dir = SAVE_DIR
        model.save_pretrained(save_dir)
        tokenizer.save_pretrained(save_dir)
        print(f"[INFO] LoRA adapter saved to {save_dir}")
        
        return result
        
    except Exception as e:
        print(f"[ERROR] Training failed: {e}")
        return None


# ===============================
# 6. TEST INFERENCE
# ===============================
def test_inference(model, tokenizer, test_text):
    """Test model sau khi training"""
    prompt = f"### Input:\n{test_text}\n\n### Summary:\n"
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            early_stopping=True
        )
    
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Trích xuất phần tóm tắt
    if "### Summary:" in full_response:
        summary = full_response.split("### Summary:")[-1].strip()
    else:
        summary = full_response
    
    print(f"[TEST] Input length: {len(test_text)} chars")
    print(f"[TEST] Input preview: {test_text[:100]}...")
    print(f"[TEST] Generated summary: {summary}")
    return summary


# ===============================
# 7. ALTERNATIVE APPROACH - Dynamic Padding
# ===============================
def load_and_prepare_data_dynamic(tokenizer, split_ratio="train[:3%]"):
    """Alternative approach với dynamic padding"""
    print("[INFO] Loading dataset with dynamic padding approach...")
    ds = load_dataset("OpenHust/vietnamese-summarization", split=split_ratio)
    
    text_column = 'Document'
    summary_column = 'Summary'

    def preprocess_function(examples):
        texts = []
        
        for i in range(len(examples[text_column])):
            input_text = examples[text_column][i]
            summary_text = examples[summary_column][i]
            
            full_text = f"### Input:\n{input_text}\n\n### Summary:\n{summary_text}{tokenizer.eos_token}"
            texts.append(full_text)
        
        # Tokenize KHÔNG padding ở đây
        tokenized = tokenizer(
            texts,
            max_length=1024,
            truncation=True,
            padding=False,  # Không padding ở preprocessing
            return_tensors=None
        )
        
        tokenized["labels"] = tokenized["input_ids"].copy()
        return tokenized

    tokenized_ds = ds.map(
        preprocess_function,
        batched=True,
        batch_size=1000,
        remove_columns=ds.column_names,
        num_proc=1
    )

    return tokenized_ds


# ===============================
# 8. MAIN - FLEXIBLE
# ===============================
# if __name__ == "__main__":
#     print("🚀 Starting training...")
#     model, tokenizer = load_model()
    
#     # Thử cả hai cách tiếp cận
#     print("🔧 Testing data preprocessing...")
    
#     # Cách 1: Padding trong preprocessing
#     try:
#         tokenized_ds = load_and_prepare_data(tokenizer, "train[:1%]")
#         print("✅ Approach 1 (pre-padding) successful")
#     except Exception as e:
#         print(f"❌ Approach 1 failed: {e}")
#         # Thử cách 2
#         print("🔄 Trying approach 2 (dynamic padding)...")
#         tokenized_ds = load_and_prepare_data_dynamic(tokenizer, "train[:1%]")
#         print("✅ Approach 2 (dynamic padding) successful")
    
#     # Test inference trước training
#     print("\n🧪 Testing inference before training...")
#     test_sample = "Việt Nam là một quốc gia nằm ở khu vực Đông Nam Á, có bờ biển dài và nền văn hóa phong phú."
#     test_inference(model, tokenizer, test_sample)
    
#     training_args = setup_training_args(NUM_EPOCHS=2)
#     result = train_model(model, tokenizer, tokenized_ds, training_args)
    
#     if result:
#         # Test inference sau training
#         print("\n🧪 Testing inference after training...")
#         test_inference(model, tokenizer, test_sample)
#     else:
#         print("❌ Training failed, skipping inference test")

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model, tokenizer = load_model()

[INFO] Loading model: Qwen/Qwen3-0.6B


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


[INFO] LoRA layers attached successfully.
trainable params: 4,587,520 || all params: 600,637,440 || trainable%: 0.7638


In [6]:
# Test inference trước khi train
print("\n[INFO] Testing inference before training...")
test_sample = "Việt Nam là một quốc gia nằm ở khu vực Đông Nam Á, có bờ biển dài và nền văn hóa phong phú. Đất nước này có lịch sử lâu đời và đã trải qua nhiều giai đoạn phát triển quan trọng."
test_inference(model, tokenizer, test_sample)

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



[INFO] Testing inference before training...
[TEST] Input length: 177 chars
[TEST] Input preview: Việt Nam là một quốc gia nằm ở khu vực Đông Nam Á, có bờ biển dài và nền văn hóa phong phú. Đất nước...
[TEST] Generated summary: Tóm tắt nội dung chính của bài viết.

---

**Output Format:**
- Start with `Summary:` followed by the summary in Vietnamese.
- If there are multiple sentences or paragraphs, ensure that they follow a continuous format.

---

**Example:**
- Summary: "Vietnam is located in Southeast Asia, with long and wide coastlines and rich cultural traditions."

---
Okay, let's see. The user provided a passage about Vietnam's geographical location and history, and wants me to summarize it into one sentence. Let me check again.

The original input is:

"Việt Nam là một quốc gia nằm ở khu vực Đông Nam Á, có bờ biển dài và nền văn hóa phong phú. Đất nước này có lịch sử


'Tóm tắt nội dung chính của bài viết.\n\n---\n\n**Output Format:**\n- Start with `Summary:` followed by the summary in Vietnamese.\n- If there are multiple sentences or paragraphs, ensure that they follow a continuous format.\n\n---\n\n**Example:**\n- Summary: "Vietnam is located in Southeast Asia, with long and wide coastlines and rich cultural traditions."\n\n---\nOkay, let\'s see. The user provided a passage about Vietnam\'s geographical location and history, and wants me to summarize it into one sentence. Let me check again.\n\nThe original input is:\n\n"Việt Nam là một quốc gia nằm ở khu vực Đông Nam Á, có bờ biển dài và nền văn hóa phong phú. Đất nước này có lịch sử'

In [7]:
tokenized_ds = load_and_prepare_data(tokenizer)

[INFO] Loading dataset...
[INFO] Dataset size: 2237 samples

📊 TOKENIZATION STATS:
Total tokens - Avg: 1024.0, Max: 1024, Min: 1024
Unique sequence lengths: {1024}


In [8]:
print(tokenized_ds)
print(len(tokenized_ds))


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2237
})
2237


In [9]:
training_args = setup_training_args(OUTPUT_DIR="./runs/qwen3-0.6B-summary", NUM_EPOCHS=2)

In [10]:
train_model(model, tokenizer, tokenized_ds, training_args)

C:\Users\Admin\AppData\Local\Temp\ipykernel_7960\3896901753.py:156: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


[INFO] Initializing Trainer...
[INFO] Start training with LoRA...
[DEBUG] Dataset info:
  - Number of samples: 2237
  - Features: {'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}
[DEBUG] First sample structure:
  - input_ids: <class 'list'>, length: 1024
  - attention_mask: <class 'list'>, length: 1024
  - labels: <class 'list'>, length: 1024
[DEBUG] Testing data collator with 2 samples...
[DEBUG] Collated batch shapes:
  - input_ids: torch.Size([2, 1024])
  - attention_mask: torch.Size([2, 1024])
  - labels: torch.Size([2, 1024])
[DEBUG] Checking dataloader...
[DEBUG] Batch shapes:
  - Input IDs: torch.Size([2, 1024])
  - Labels: torch.Size([2, 1024])
  - Attention mask: torch.Size([2, 1024])
[DEBUG] Sample input (first 30 tokens):
### Input:
Theo trang Technology Review , thành tựu nghiên cứu mới của các nhà khoa học thuộc Đại học Yale ( Mỹ ) được


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,3.524200
20,3.417000
30,3.247200
40,3.022100
50,2.918200
60,2.935400
70,2.876100
80,2.830000
90,2.820000
100,2.836400


[INFO] LoRA adapter saved to ./runs/qwen3-0.6B-summarization-lora


TrainOutput(global_step=560, training_loss=2.7229151453290665, metrics={'train_runtime': 3437.6932, 'train_samples_per_second': 1.301, 'train_steps_per_second': 0.163, 'total_flos': 1.2233785114361856e+16, 'train_loss': 2.7229151453290665, 'epoch': 2.0})

## TEST BASE

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model nguyên bản
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

text1 = """Ẩm thực Việt Nam rất đa dạng và phong phú, với nhiều món ăn đặc trưng theo từng vùng miền. 
Miền Bắc nổi tiếng với phở, bún chả, miền Trung có bún bò Huế, cao lầu, còn miền Nam có hủ tiếu, cơm tấm. 
Các món ăn Việt Nam thường chú trọng sự cân bằng giữa các vị chua, cay, mặn, ngọt và sử dụng nhiều rau thơm, gia vị tươi."""

prompt = f"Văn bản: {text1}\nTóm tắt:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Kết quả model nguyên bản:", result.split("Tóm tắt:")[-1].strip())

Some parameters are on the meta device because they were offloaded to the cpu.


Kết quả model nguyên bản: - Các món ăn ở miền Bắc có đặc điểm gì?
- Các món ăn ở miền Nam có đặc điểm gì?
- Các món ăn ở miền Trung có đặc điểm gì?
- Những món ăn nổi tiếng nhất ở mỗi miền nào?

**Tóm tắt ngắn gọn**:
- Miền Bắc nổi tiếng với phở, bún chả, bún bò Huế, cao lầu.
- Miền Nam nổi tiếng với hủ tiếu, cơm tấm


## TEST SFT

In [ ]:
# Code test nhanh
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load model
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", device_map="auto")
model = PeftModel.from_pretrained(model, "./runs/qwen3-0.6B-summarization-lora")
tokenizer = AutoTokenizer.from_pretrained("./runs/qwen3-0.6B-summarization-lora")

In [7]:
text1 = """Ẩm thực Việt Nam rất đa dạng và phong phú, với nhiều món ăn đặc trưng theo từng vùng miền. 
Miền Bắc nổi tiếng với phở, bún chả, miền Trung có bún bò Huế, cao lầu, còn miền Nam có hủ tiếu, cơm tấm. 
Các món ăn Việt Nam thường chú trọng sự cân bằng giữa các vị chua, cay, mặn, ngọt và sử dụng nhiều rau thơm, gia vị tươi."""
prompt = f"Văn bản: {text1}\nTóm tắt:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Kết quả:", result.split("Tóm tắt:")[-1].strip())

Kết quả: Các món ăn Việt Nam có nhiều đặc trưng theo từng vùng miền, đặc trưng cho sự cân bằng giữa các vị chua, cay, mặn, ngọt và sử dụng nhiều rau thơm, gia vị tươi. 

Trình bày : Ở Việt Nam, món ăn thường được chế biến bằng nhiều phương pháp khác nhau. Trong đó, có người dùng nhiều món ăn có chung một món ăn. Một số món ăn được chế biến bởi người dân Việt


In [8]:
text2 = """Nền kinh tế Việt Nam trong những năm gần đây đã có sự tăng trưởng ấn tượng với tốc độ tăng trưởng GDP trung bình khoảng 6-7% mỗi năm. 
Các ngành công nghiệp chủ lực bao gồm sản xuất, công nghệ thông tin, nông nghiệp và du lịch. 
Việt Nam đã ký kết nhiều hiệp định thương mại tự do quan trọng như CPTPP, EVFTA, giúp mở rộng thị trường xuất khẩu."""
prompt = f"Văn bản: {text2}\nTóm tắt:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Kết quả:", result.split("Tóm tắt:")[-1].strip())

Kết quả: Việt Nam có tốc độ tăng trưởng GDP trung bình 6-7% mỗi năm với các ngành công nghiệp chủ lực. Các hiệp định thương mại tự do mở rộng thị trường xuất khẩu. 

Kết luận: Tăng trưởng GDP của Việt Nam trong năm 2020 được đánh giá là tốt hơn năm 2019 và 2018. Tăng trưởng đạt 5,3% trong năm 2019, 5,4%


In [9]:
text3 = """Việt Nam sở hữu nhiều danh lam thắng cảnh nổi tiếng thu hút du khách quốc tế. 
Ở miền Bắc có Vịnh Hạ Long - di sản thiên nhiên thế giới, Sa Pa với những thửa ruộng bậc thang. 
Miền Trung có cố đô Huế, phố cổ Hội An, còn miền Nam có đồng bằng sông Cửu Long với các khu chợ nổi đặc sắc và hệ thống sông ngòi chằng chịt."""

prompt = f"Văn bản: {text3}\nTóm tắt:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Kết quả:", result.split("Tóm tắt:")[-1].strip())

Kết quả: Việt Nam có 8 di sản thiên nhiên thế giới và 3 di sản văn hóa thế giới. Đa dạng sinh học, khí hậu và địa hình đặc trưng của Việt Nam đã tạo nên nhiều điểm đến hấp dẫn. Du khách quốc tế đã đến nhiều nơi với nhiều chuyến du lịch. Đa dạng sinh học, khí hậu và địa hình đặc trưng của Việt Nam đã tạo nên nhiều điểm đến hấp dẫn. Du khách quốc tế đã đến nhiều nơi với nhiều chuyến du lịch.
